<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="../../index.ipynb">
        <img src="../../static/images/epyklogo_whole_small.png">
    </a>
</div>

# Add bespoke package

It is possible to add a custom package to your project without creating a new project and package.

It is possible to extend the list of underlying external packages using the Import module in Epyk.
This module is in charge of keeping the scope of external packages (Javascript and CSS) with the current version.

The version of the external packages is fixed as it is important to ensure that m

## Extend the Imports

In [1]:
from epyk.core.js import Imports

# Extend the current list of packages to include Google charts as a reference
Imports.extend("google-chart", [('loader.js',  'charts/')], version=None, cdnjs_url="https://www.gstatic.com", required=None)

    

## Adding components

Next step is to create the new component using this new alias

The class must look like a classic HTML page



In [4]:
from epyk.core.html import Html

# Create the bespoke HTML component
class Chart(Html.Html):
  # Link this component to the external Javascript module
  __reqJs = ["google-chart"]

  def __init__(self, report, vals, htmlCode=None, width=None, height=None, options=None, profile=None):
    super(Chart, self).__init__(report, vals, htmlCode=htmlCode, css_attrs={"width": width, "height": height})
    self._jsStyles = options or {}

  @property
  def _js__builder__(self):
    # Create the generic build function for those object
    # Those generic Javascrip builder are receiving the below parameters
    #   - htmlObj: The HTML dom object
    #   - data: The value passed in the vals variable
    #   - options: The self._jsStyles object
    # This function is generic and will be used by all the different object created
    # This Js fragment will be used in the refresh and build method
    return '''
      var chart = new google.visualization.AreaChart(htmlObj);
      chart.draw(google.visualization.arrayToDataTable(data), options) '''

  def __str__(self):
    # Load the Google module
    # This module require some specific loading function
    self._report._props.setdefault('js', {}).setdefault("builders", []).append("google.charts.load('current', {'packages':['corechart']})")
    self._report._props.setdefault('js', {}).setdefault("builders", []).append("google.charts.setOnLoadCallback( (function(){return %s}) )" % self.refresh())
    # The HTML component
    return '<div %s></div>' % self.get_attrs(pyClassNames=self.style.get_classes())



## Report use

Then this new component can be used directly by passing the class from the **bespoke** function in the ui property. The following parameters to this function are all the parameter defined in the __init__ method in the above class

In [7]:
from epyk.core.Page import Report

# Create a basic report object
rptObj = Report()

# The input data from https://developers.google.com/chart/interactive/docs/gallery/areachart
data = [['Year', 'Sales', 'Expenses'], ['2013', 1000, 400], ['2014', 1170, 460], ['2015', 660, 1120], ['2016', 1030, 540]]

# Create an object on the Javascript side
rptObj.ui.bespoke(Chart, vals=data)

rptObj.outs.jupyter()

[Go back to the tutorials page](../../tutorials.ipynb)

<hr>
Do not forget that this is a collaborative framework so do not hesitate to give feedbacks and like the different repository to get more visbility.

Also any help is more than welcome !